# BITCOIN4Traders - Google Colab Training

**Anleitung:**
1. Gehe zu `Runtime > Change runtime type` und wähle **GPU (T4)**
2. Führe alle Zellen der Reihe nach aus
3. Das Modell wird automatisch auf Google Drive gespeichert
4. Bei Unterbrechung: Zelle 1-4 erneut ausführen, dann Resume-Zelle

---

## Zelle 0b: Error-Handler & Hilferuf (Auto-Repair)
> **Automatisch eingefügt.** Sendet Fehler an Linux-PC für Selbstheilung.

In [ ]:
# ═══════════════════════════════════════════════════════════════
# AUTO-REPAIR: Error-Handler & Hilferuf zum Linux-PC
# ═══════════════════════════════════════════════════════════════
# Sendet Fehler automatisch an den Linux-Master.
# Kanal 1: HTTP (schnell, direkter Weg)
# Kanal 2: Google Drive (Fallback, kein offener Port nötig)
# ═══════════════════════════════════════════════════════════════

import sys
import json
import traceback
import requests
from datetime import datetime
from pathlib import Path

# ─── Konfiguration ───────────────────────────────────────────────
NOTEBOOK_ID     = 'BITCOIN4Traders_Colab_v1'
LINUX_IP        = ''           # z.B. '192.168.1.100' oder ngrok-URL
LINUX_PORT      = 5001
LINUX_API_TOKEN = ''           # Von: python3 infrastructure/monitor/listener.py setup
DRIVE_FOLDER_ID = ''           # Google Drive Ordner-ID (Fallback)

# ─── Hilferuf-Funktion ───────────────────────────────────────────
def send_help(error_msg: str, error_type: str = 'Exception',
              extra: dict = None):
    """Sendet Fehlerbericht an Linux-PC via HTTP oder Drive."""
    payload = {
        'notebook_id':   NOTEBOOK_ID,
        'error_type':    error_type,
        'error_message': str(error_msg)[:500],
        'stacktrace':    traceback.format_exc()[:2000],
        'timestamp':     datetime.now().isoformat(),
        'extra':         extra or {},
    }

    # Kanal 1: HTTP
    if LINUX_IP and LINUX_API_TOKEN:
        try:
            r = requests.post(
                f'http://{LINUX_IP}:{LINUX_PORT}/report_error',
                json=payload,
                headers={'X-API-Token': LINUX_API_TOKEN},
                timeout=10
            )
            if r.status_code == 200:
                print(f'✅ Hilferuf empfangen (HTTP): {r.json()}')
                return
        except Exception as e:
            print(f'⚠️ HTTP fehlgeschlagen ({e}) - versuche Drive...')

    # Kanal 2: Google Drive (Fallback)
    _send_help_via_drive(payload)


def _send_help_via_drive(payload: dict):
    """Schreibt Fehlerbericht in Google Drive (Fallback)."""
    if not DRIVE_FOLDER_ID:
        print('⚠️ DRIVE_FOLDER_ID nicht gesetzt - Hilferuf konnte nicht gesendet werden')
        return
    try:
        from google.colab import auth
        from googleapiclient.discovery import build
        from googleapiclient.http import MediaFileUpload
        import tempfile
        auth.authenticate_user()
        from google.auth import default
        creds, _ = default()
        service = build('drive', 'v3', credentials=creds)
        with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as f:
            json.dump(payload, f)
            tmp = f.name
        media = MediaFileUpload(tmp, mimetype='application/json')
        service.files().create(
            body={'name': 'colab_error_report.json', 'parents': [DRIVE_FOLDER_ID]},
            media_body=media
        ).execute()
        print('✅ Hilferuf via Drive gesendet')
    except Exception as e:
        print(f'❌ Drive-Hilferuf fehlgeschlagen: {e}')


# ─── Status-Update (periodisch) ──────────────────────────────────
def report_status(step: int, reward: float, extra: dict = None):
    """Sendet Heartbeat mit Trainings-Fortschritt."""
    if not DRIVE_FOLDER_ID:
        return
    try:
        status = {
            'timestamp':    datetime.now().isoformat(),
            'notebook_id':  NOTEBOOK_ID,
            'training_step': step,
            'last_reward':  round(float(reward), 4),
            'status':       'training',
            **(extra or {}),
        }
        # Lokal in Drive schreiben
        status_path = '/content/drive/MyDrive/BITCOIN4Traders/colab_status.json'
        with open(status_path, 'w') as f:
            json.dump(status, f)
    except Exception:
        pass  # Niemals Training unterbrechen wegen Status-Update


print('✅ Error-Handler geladen')
print(f'   Notebook-ID: {NOTEBOOK_ID}')
print(f'   HTTP-Kanal:  {LINUX_IP}:{LINUX_PORT} ', '✅' if LINUX_IP else '⚠️ nicht konfiguriert')
print(f'   Drive-Kanal: ', '✅' if DRIVE_FOLDER_ID else '⚠️ nicht konfiguriert')


## Zelle 1: GPU prüfen

In [ ]:
import torch

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f'GPU verfügbar: {gpu_name}')
    print(f'GPU Speicher: {gpu_mem:.1f} GB')
    DEVICE = 'cuda'
else:
    print('WARNUNG: Keine GPU gefunden! Gehe zu Runtime > Change runtime type > GPU')
    DEVICE = 'cpu'

print(f'Verwende Device: {DEVICE}')

## Zelle 2: Google Drive mounten (für persistente Speicherung)

In [ ]:
from google.colab import drive
import os

# Google Drive mounten
drive.mount('/content/drive')

# Projektordner auf Drive erstellen
DRIVE_PROJECT_DIR = '/content/drive/MyDrive/BITCOIN4Traders'
DRIVE_MODEL_DIR = f'{DRIVE_PROJECT_DIR}/models'
DRIVE_DATA_DIR = f'{DRIVE_PROJECT_DIR}/data'
DRIVE_LOG_DIR = f'{DRIVE_PROJECT_DIR}/logs'

os.makedirs(DRIVE_MODEL_DIR, exist_ok=True)
os.makedirs(DRIVE_DATA_DIR, exist_ok=True)
os.makedirs(DRIVE_LOG_DIR, exist_ok=True)

print(f'Google Drive gemountet.')
print(f'Modelle werden gespeichert in: {DRIVE_MODEL_DIR}')
print(f'Daten werden gespeichert in: {DRIVE_DATA_DIR}')

## Zelle 3: Repository klonen / Projekt hochladen

In [ ]:
import os
import shutil

PROJECT_DIR = '/content/BITCOIN4Traders'
REPO_URL = 'https://github.com/juancarlosrial76-code/BITCOIN4Traders.git'

# Wenn Ordner existiert aber kein gültiges Git-Repo ist -> löschen
if os.path.exists(PROJECT_DIR) and not os.path.exists(os.path.join(PROJECT_DIR, '.git')):
    print(f'Ungültiger Projektordner gefunden - wird gelöscht...')
    shutil.rmtree(PROJECT_DIR)

if not os.path.exists(PROJECT_DIR):
    print('Klone Projekt von GitHub...')
    !git clone {REPO_URL} {PROJECT_DIR}
    print('Klon abgeschlossen!')
else:
    print('Projekt vorhanden - aktualisiere...')
    !git -C {PROJECT_DIR} pull
    print('Aktualisiert!')

# Prüfung
src_ok = os.path.exists(os.path.join(PROJECT_DIR, 'src', 'data', 'ccxt_loader.py'))
print(f'\nProjektstruktur OK: {src_ok}')
if not src_ok:
    raise RuntimeError("Klon fehlgeschlagen! Bitte Runtime neu starten und erneut versuchen.")

os.chdir(PROJECT_DIR)
print(f'Bereit: {PROJECT_DIR}')


## Zelle 4: Dependencies installieren

In [ ]:
%%time
print('Installiere Dependencies...')

!pip install -q ccxt loguru pyarrow pandas numpy scipy gymnasium stable-baselines3 ta yfinance numba hmmlearn scikit-learn pyyaml pydantic python-dotenv tqdm joblib matplotlib plotly omegaconf

# Sicherstellen dass ccxt wirklich installiert ist
import importlib
for pkg in ['ccxt', 'loguru', 'pyarrow', 'gymnasium', 'omegaconf']:
    try:
        importlib.import_module(pkg)
        print(f'  OK: {pkg}')
    except ImportError:
        print(f'  FEHLT: {pkg} - installiere nochmals...')
        import subprocess
        subprocess.run(['pip', 'install', '-q', pkg], check=True)

print('Installation abgeschlossen!')


## Zelle 5: Python-Pfad setzen

In [ ]:
import sys
import os

PROJECT_DIR = '/content/BITCOIN4Traders'
SRC_DIR = os.path.join(PROJECT_DIR, 'src')

# Pfade hinzufügen
for path in [PROJECT_DIR, SRC_DIR]:
    if path not in sys.path:
        sys.path.insert(0, path)

# In Projektordner wechseln
os.chdir(PROJECT_DIR)
print(f'Arbeitsverzeichnis: {os.getcwd()}')
print(f'Python-Pfad enthält: {SRC_DIR}')

# Notwendige Verzeichnisse erstellen
dirs = [
    'data/cache',
    'data/processed', 
    'data/models/adversarial',
    'logs/training'
]
for d in dirs:
    os.makedirs(d, exist_ok=True)

print('Verzeichnisse erstellt.')

## Zelle 6: Daten von Drive laden oder herunterladen

In [ ]:
import shutil
import os

DRIVE_DATA_DIR = '/content/drive/MyDrive/BITCOIN4Traders/data'
LOCAL_CACHE_DIR = '/content/BITCOIN4Traders/data/cache'

# Prüfen ob gecachte Daten auf Drive vorhanden
drive_cache_files = []
if os.path.exists(DRIVE_DATA_DIR):
    drive_cache_files = [f for f in os.listdir(DRIVE_DATA_DIR) if f.endswith('.parquet')]

if drive_cache_files:
    print(f'Lade gecachte Daten von Drive: {drive_cache_files}')
    for fname in drive_cache_files:
        src = os.path.join(DRIVE_DATA_DIR, fname)
        dst = os.path.join(LOCAL_CACHE_DIR, fname)
        shutil.copy2(src, dst)
        print(f'  Kopiert: {fname}')
    print('Daten erfolgreich von Drive geladen!')
else:
    print('Keine gecachten Daten auf Drive gefunden.')
    print('Daten werden beim Training von Binance heruntergeladen.')
    print('(Dies dauert einige Minuten beim ersten Start)')

## Zelle 7: Training-Konfiguration

In [ ]:
import torch

# ===== TRAINING-EINSTELLUNGEN =====
# Diese Werte kannst du anpassen

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Datensatz
SYMBOL = 'BTC/USDT'        # Handelspaar
TIMEFRAME = '1h'            # Zeitrahmen: 1m, 5m, 15m, 1h, 4h, 1d
START_DATE = '2022-01-01'   # Startdatum (mehr Daten = besseres Training)
END_DATE = None              # None = bis heute
EXCHANGE = 'kucoin'

# Training
N_ITERATIONS = 500           # Anzahl Trainingsiterationen
STEPS_PER_ITER = 2048        # Schritte pro Iteration
SAVE_FREQUENCY = 25          # Speichern alle N Iterationen (öfter als Standard)

# Checkpoint (für Resume)
RESUME_CHECKPOINT = None     # Pfad zu Checkpoint, z.B. 'data/models/adversarial/checkpoint_iter_100.pth'

print('Konfiguration:')
print(f'  Device:     {DEVICE}')
print(f'  Symbol:     {SYMBOL}')
print(f'  Timeframe:  {TIMEFRAME}')
print(f'  Start:      {START_DATE}')
print(f'  Iterationen: {N_ITERATIONS}')
print(f'  Save alle:  {SAVE_FREQUENCY} Iterationen')

## Zelle 8: Daten laden & Features berechnen

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from loguru import logger
import sys, os, importlib.util, subprocess

PROJECT_DIR = '/content/BITCOIN4Traders'
SRC_DIR = os.path.join(PROJECT_DIR, 'src')
ccxt_loader_path = os.path.join(SRC_DIR, 'data', 'ccxt_loader.py')

# Logging
logger.remove()
logger.add(sys.stdout, format="{time:HH:mm:ss} | {level} | {message}", level="INFO")
# Log-Datei mit Rotation: max 50 MB, max 3 Backups -> verhindert unkontrollierten Speicheranstieg
logger.add(
    '/content/BITCOIN4Traders/logs/training/training.log',
    format="{time:HH:mm:ss} | {level} | {message}",
    level="INFO",
    rotation="50 MB",
    retention=3,
    compression="gz",
)

# Absolute Pfade
cache_dir = Path(os.path.join(PROJECT_DIR, 'data', 'cache'))
processed_dir = Path(os.path.join(PROJECT_DIR, 'data', 'processed'))
cache_dir.mkdir(parents=True, exist_ok=True)
processed_dir.mkdir(parents=True, exist_ok=True)

def load_via_yfinance(symbol, start_date, end_date, cache_dir):
    """Lädt BTC Daten via Yahoo Finance - kein Geo-Block."""
    import yfinance as yf
    # BTC/USDT -> BTC-USD für Yahoo Finance
    yf_symbol = symbol.replace('/', '-').replace('USDT', 'USD')
    logger.info(f'Lade {yf_symbol} von Yahoo Finance...')
    df = yf.download(yf_symbol, start=start_date, end=end_date, interval='1h', progress=False, auto_adjust=True)
    if df.empty:
        raise ValueError(f'Keine Daten von Yahoo Finance für {yf_symbol}')
    # Spalten anpassen
    df.columns = [c.lower() for c in df.columns]
    df = df[['open', 'high', 'low', 'close', 'volume']]
    df.index.name = 'timestamp'
    df.index = pd.to_datetime(df.index, utc=True).tz_localize(None)
    # Cache speichern
    cache_path = cache_dir / f'BTC_USDT_1h_yfinance.parquet'
    df.to_parquet(cache_path, engine='pyarrow', compression='snappy')
    logger.success(f'Yahoo Finance: {len(df)} Candles geladen')
    return df

def load_via_ccxt(exchange_id, symbol, timeframe, start_date, end_date, cache_dir, processed_dir):
    """Lädt Daten via CCXT."""
    spec = importlib.util.spec_from_file_location("ccxt_loader", ccxt_loader_path)
    mod = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(mod)
    config = mod.DataLoaderConfig(
        exchange_id=exchange_id,
        exchange_type='spot',
        rate_limit_ms=200,
        cache_dir=cache_dir,
        processed_dir=processed_dir,
        compression='snappy',
    )
    loader = mod.CCXTDataLoader(config)
    return loader.download_and_cache(
        symbol=symbol, timeframe=timeframe,
        start_date=start_date, end_date=end_date,
        force_refresh=False,
    )

# Cache prüfen
cached_files = list(cache_dir.glob('*.parquet'))
if cached_files:
    logger.info(f'Lade gecachte Daten: {cached_files[0]}')
    price_data = pd.read_parquet(cached_files[0])
    logger.success(f'Geladen: {len(price_data)} Candles')
else:
    # Versuche CCXT Exchanges ohne Geo-Block
    exchanges_to_try = ['kucoin', 'bybit', 'okx', 'gateio']
    price_data = None

    for exchange_id in exchanges_to_try:
        try:
            logger.info(f'Versuche {exchange_id}...')
            price_data = load_via_ccxt(
                exchange_id, SYMBOL, TIMEFRAME,
                START_DATE, END_DATE,
                cache_dir, processed_dir
            )
            logger.success(f'{exchange_id}: Erfolgreich!')
            break
        except Exception as e:
            logger.warning(f'{exchange_id} fehlgeschlagen: {str(e)[:80]}')
            continue

    # Fallback: Yahoo Finance
    if price_data is None:
        logger.info('Alle CCXT Exchanges fehlgeschlagen - nutze Yahoo Finance als Fallback...')
        try:
            subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'yfinance'], check=True)
            price_data = load_via_yfinance(SYMBOL, START_DATE, END_DATE, cache_dir)
        except Exception as e:
            raise RuntimeError(f'Alle Datenquellen fehlgeschlagen: {e}')

    # Drive sichern
    if os.path.exists('/content/drive/MyDrive'):
        import shutil
        drive_dir = '/content/drive/MyDrive/BITCOIN4Traders/data'
        os.makedirs(drive_dir, exist_ok=True)
        for f in cache_dir.glob('*.parquet'):
            shutil.copy2(str(f), os.path.join(drive_dir, f.name))
            logger.info(f'Drive: {f.name} gespeichert')

print(f'\nDatensatz: {len(price_data)} Zeilen')
print(f'Zeitraum: {price_data.index[0]} bis {price_data.index[-1]}')
price_data.head()


## Zelle 9: Feature Engineering

In [ ]:
from features.feature_engine import FeatureEngine, FeatureConfig

logger.info('Feature Engineering...')

feature_config = FeatureConfig(
    volatility_window=20,
    ou_window=20,
    rolling_mean_window=20,
    use_log_returns=True,
    scaler_type='standard',
    save_scaler=True,
    scaler_path=processed_dir,
    dropna_strategy='rolling',
    min_valid_rows=1000,
)

engine = FeatureEngine(feature_config)

# Chronologischer Split: 70% Train, 15% Val, 15% Test
n = len(price_data)
train_idx = int(n * 0.70)
val_idx = int(n * 0.85)

train_data = price_data.iloc[:train_idx]
val_data = price_data.iloc[train_idx:val_idx]
test_data = price_data.iloc[val_idx:]

logger.info(f'Split: Train={len(train_data)}, Val={len(val_data)}, Test={len(test_data)}')

# Fit NUR auf Trainingsdaten (kein Data Leakage!)
logger.info('Fit FeatureEngine auf Trainingsdaten...')
train_features = engine.fit_transform(train_data)

logger.info('Transformiere Val und Test...')
val_features = engine.transform(val_data)
test_features = engine.transform(test_data)

# Indizes angleichen
common_train = train_data.index.intersection(train_features.index)
train_price = train_data.loc[common_train]
train_feat = train_features.loc[common_train]

logger.success(f'Features berechnet: {train_feat.shape[1]} Features, {len(train_price)} Trainingssamples')
print(f'Feature-Spalten: {list(train_feat.columns[:5])}...')

## Zelle 10: Environment erstellen

In [ ]:
from environment.config_integrated_env import ConfigIntegratedTradingEnv
from environment.config_system import EnvironmentConfig, load_environment_config_from_yaml

config_path = Path('config/environment/realistic_env.yaml')

if config_path.exists():
    env_config = load_environment_config_from_yaml(str(config_path))
    logger.info('Environment-Config geladen')
else:
    env_config = EnvironmentConfig()
    logger.warning('Verwende Standard-Config')

env = ConfigIntegratedTradingEnv(train_price, train_feat, env_config)

logger.success('Trading Environment erstellt')
print(f'Observation Space: {env.observation_space.shape}')
print(f'Action Space: {env.action_space.n}')

## Zelle 11: Trainer erstellen

In [ ]:
from agents.ppo_agent import PPOConfig
from training.adversarial_trainer import AdversarialTrainer, AdversarialConfig

state_dim = env.observation_space.shape[0]
n_actions = env.action_space.n

# Trader (optimiert für Profit)
trader_config = PPOConfig(
    state_dim=state_dim,
    hidden_dim=128,
    n_actions=n_actions,
    actor_lr=3e-4,
    critic_lr=1e-3,
    gamma=0.99,
    gae_lambda=0.95,
    clip_epsilon=0.2,
    n_epochs=10,
    batch_size=64,
    use_recurrent=True,
    rnn_type='GRU',
    entropy_coef=0.01,
    value_loss_coef=0.5,
    max_grad_norm=0.5,
    target_kl=0.01,
)

# Adversary (erschafft schwierige Szenarien)
adversary_config = PPOConfig(
    state_dim=state_dim,
    hidden_dim=128,
    n_actions=n_actions,
    actor_lr=1e-4,
    critic_lr=5e-4,
    gamma=0.99,
    gae_lambda=0.95,
    clip_epsilon=0.2,
    n_epochs=10,
    batch_size=64,
    use_recurrent=True,
    rnn_type='GRU',
    entropy_coef=0.02,
)

# Training-Konfiguration
training_config = AdversarialConfig(
    n_iterations=N_ITERATIONS,
    steps_per_iteration=STEPS_PER_ITER,
    trader_config=trader_config,
    adversary_config=adversary_config,
    adversary_start_iteration=100,
    adversary_strength=0.1,
    save_frequency=SAVE_FREQUENCY,
    log_frequency=10,
    checkpoint_dir='data/models/adversarial',
)

trainer = AdversarialTrainer(env, training_config, device=DEVICE)

logger.success('Trainer erstellt')
print(f'State dim: {state_dim}, Actions: {n_actions}')
print(f'Device: {DEVICE}')
print(f'Iterationen: {N_ITERATIONS}')

## Zelle 12: [Optional] Von Checkpoint weitermachen

In [ ]:
from loguru import logger
import os
import shutil

DRIVE_MODEL_DIR = '/content/drive/MyDrive/BITCOIN4Traders/models'
LOCAL_MODEL_DIR = '/content/BITCOIN4Traders/data/models/adversarial'

# Prüfe ob Checkpoints auf Drive vorhanden
drive_checkpoints = []
if os.path.exists(DRIVE_MODEL_DIR):
    drive_checkpoints = sorted(
        [f for f in os.listdir(DRIVE_MODEL_DIR) if f.endswith('.pth')]
    )

if drive_checkpoints:
    latest = drive_checkpoints[-1]
    src = os.path.join(DRIVE_MODEL_DIR, latest)
    dst = os.path.join(LOCAL_MODEL_DIR, latest)
    shutil.copy2(src, dst)
    
    logger.info(f'Lade Checkpoint: {latest}')
    try:
        trainer.load_checkpoint(dst)
        logger.success(f'Checkpoint geladen: {latest}')
    except Exception as e:
        logger.error(f'Fehler beim Laden: {e}')
else:
    logger.info('Kein Checkpoint gefunden - starte Training von Anfang an')

# Manuell einen Checkpoint angeben:
# RESUME_CHECKPOINT = 'data/models/adversarial/checkpoint_iter_200.pth'
# if RESUME_CHECKPOINT and os.path.exists(RESUME_CHECKPOINT):
#     trainer.load_checkpoint(RESUME_CHECKPOINT)

## Zelle 13: Auto-Save Callback einrichten

In [ ]:
import os
import shutil
import glob

def sync_models_to_drive():
    """Kopiert alle lokalen Checkpoints auf Google Drive."""
    local_dir = '/content/BITCOIN4Traders/data/models/adversarial'
    drive_dir = '/content/drive/MyDrive/BITCOIN4Traders/models'
    
    checkpoints = glob.glob(os.path.join(local_dir, '*.pth'))
    for cp in checkpoints:
        fname = os.path.basename(cp)
        dst = os.path.join(drive_dir, fname)
        shutil.copy2(cp, dst)
    
    if checkpoints:
        print(f'Drive sync: {len(checkpoints)} Checkpoint(s) gespeichert')

# Test
sync_models_to_drive()
print('Auto-Save Funktion bereit.')

## Zelle 14: TRAINING STARTEN

> **Tipp:** Halte die Seite aktiv (z.B. Tab offen lassen) um Session-Timeouts zu vermeiden.

In [ ]:
import time
from loguru import logger

logger.info('=' * 60)
logger.info('TRAINING STARTET')
logger.info('=' * 60)

start_time = time.time()
_training_completed = False
_current_step = 0

try:
    trainer.train()
    _training_completed = True

except KeyboardInterrupt:
    logger.warning('Training unterbrochen (KeyboardInterrupt)')
    logger.info('Speichere aktuellen Stand...')

except RuntimeError as e:
    error_str = str(e)
    logger.error(f'RuntimeError: {error_str}')
    if 'out of memory' in error_str.lower() or 'cuda' in error_str.lower():
        import torch; torch.cuda.empty_cache()
        send_help(error_str, error_type='CUDA_ERROR',
                  extra={'step': _current_step, 'elapsed_h': round((time.time()-start_time)/3600, 2)})
    else:
        send_help(error_str, error_type='RuntimeError',
                  extra={'step': _current_step})

except MemoryError as e:
    logger.error(f'MemoryError (RAM): {e}')
    send_help(str(e), error_type='OOM',
              extra={'step': _current_step})

except Exception as e:
    import traceback
    logger.error(f'Unbekannter Fehler: {e}')
    traceback.print_exc()
    send_help(str(e), error_type='Exception',
              extra={'step': _current_step, 'elapsed_h': round((time.time()-start_time)/3600, 2)})

finally:
    elapsed = (time.time() - start_time) / 3600
    if _training_completed:
        logger.success(f'Training abgeschlossen! Dauer: {elapsed:.1f}h')
        report_status(_current_step, 0, extra={'status': 'completed', 'elapsed_h': elapsed})
    else:
        logger.info(f'Training gestoppt nach {elapsed:.1f}h')
    # Immer auf Drive speichern!
    logger.info('Synchronisiere mit Google Drive...')
    sync_models_to_drive()
    logger.success('Modell auf Drive gesichert!')


## Zelle 15: Evaluation

In [ ]:
logger.info('Evaluiere trainiertes Modell...')

try:
    metrics = trainer.evaluate(n_episodes=100)
    
    print('\n=== Evaluationsergebnisse ===')
    for key, value in metrics.items():
        if isinstance(value, float):
            print(f'  {key}: {value:.4f}')
        else:
            print(f'  {key}: {value}')

except Exception as e:
    logger.error(f'Evaluation fehlgeschlagen: {e}')
    import traceback
    traceback.print_exc()

## Zelle 16: Checkpoints auf Drive anzeigen

In [ ]:
import os

DRIVE_MODEL_DIR = '/content/drive/MyDrive/BITCOIN4Traders/models'

print('Gespeicherte Modelle auf Google Drive:')
print('=' * 50)

if os.path.exists(DRIVE_MODEL_DIR):
    files = sorted(os.listdir(DRIVE_MODEL_DIR))
    total_mb = 0
    for f in files:
        path = os.path.join(DRIVE_MODEL_DIR, f)
        size_mb = os.path.getsize(path) / 1e6
        total_mb += size_mb
        print(f'  {f:40s}  {size_mb:.1f} MB')
    print(f'\nGesamt: {len(files)} Dateien, {total_mb:.1f} MB')
else:
    print('Kein Modellordner auf Drive gefunden.')

## Zelle 17: Memory & Storage – Google Drive Sicherung

Diese Zelle richtet die automatische Speicherung auf Google Drive ein und bietet Tools zum RAM-Aufräumen.

In [ ]:
from utils.memory_utils import cleanup_memory, save_and_trim_data, print_resource_report

# RAM-Status prüfen
print_resource_report()

# Beispiel für die Nutzung:
# if len(df) > 1500:
#     df = save_and_trim_data(df, filename='trade_history.csv', max_rows=1000)
print('Memory-Utilities bereit.')

## Zelle 18: Multiverse Evolution — Darwin Engine

Multiversum-Validierung: RSI, MACD, Bollinger und EMA treten in HUNDERTEN
verschiedener Zeitlinien gegeneinander an (Monte Carlo + Stress-Szenarien).

Szenarien:
  - original       : Echte BTC-Daten unveraendert
  - flash_crash    : Plotzlicher 30% Absturz + Teilerholen
  - slow_bear      : Langsamer Baermarkt ueber alle Bars
  - sideways_hell  : Nulldrift + hohes Rauschen (zerstoert Trend-Bots)
  - parabolic_run  : Starker Bullenmarkt (testet ob Bot zu frueh aussteigt)
  - mc_bull/bear/chop (50x je): Stochastische GBM-Zeitlinien

Eliminierung: DD > 20% in IRGENDEINEM Szenario -> sofortige genetische Disqualifikation.
Champion wird automatisch auf Drive gespeichert und beim naechsten Start geladen.

In [ ]:
# ===== MULTIVERSE EVOLUTION ENGINE =====
import importlib.util, sys, os

PROJECT_DIR = '/content/BITCOIN4Traders'
SAVE_DIR = '/content/drive/MyDrive/BITCOIN4Traders/data'  # Drive-Persistenz
os.makedirs(SAVE_DIR, exist_ok=True)

_darwin_path = os.path.join(PROJECT_DIR, 'darwin_engine.py')
_spec = importlib.util.spec_from_file_location('darwin_engine', _darwin_path)
_darwin = importlib.util.module_from_spec(_spec)
_spec.loader.exec_module(_darwin)
sys.modules['darwin_engine'] = _darwin

from darwin_engine import run_multiverse, ChampionPersistence

# ===== KONFIGURATION =====
# Alle Werte aenderbar - keine hardcodierten Magic Numbers
MV_GENERATIONS    = 15    # Generationen (mehr = robusterer Champion, langsamer)
MV_POP_SIZE       = 20    # Bots pro Generation
MV_MC_SCENARIOS   = 50    # Monte Carlo Zeitlinien pro Regime (bull/bear/chop je)
MV_MAX_DD         = 0.20  # Max erlaubter Drawdown in JEDEM Szenario (20%)
MV_N_BARS         = 2000  # Basis-Datenpunkte (mind. 500)
MV_AUTO_LOAD      = True  # True = gespeicherten Champion laden falls vorhanden

# Starte die Multiversum-Evolution:
# 1. Laedt echte BTC/USDT Daten (Fallback: synthetisch)
# 2. Generiert 5 + 3*MV_MC_SCENARIOS Szenarien
# 3. Evolution: nur Ueberlebende aller Szenarien koennen sich reproduzieren
# 4. Speichert Champion automatisch auf Drive nach jeder Generation
multiverse_champion = run_multiverse(
    symbol='BTC/USDT',
    timeframe='1h',
    n_bars=MV_N_BARS,
    generations=MV_GENERATIONS,
    pop_size=MV_POP_SIZE,
    n_mc_scenarios=MV_MC_SCENARIOS,
    max_dd_threshold=MV_MAX_DD,
    auto_load_champion=MV_AUTO_LOAD,
    save_dir=SAVE_DIR,
    exchange_id='binance',
    seed=42,
)

if multiverse_champion is not None:
    print(f'\nMultiverse Champion: {multiverse_champion.name}')
    print(f'Strategie-Typ      : {type(multiverse_champion).__name__}')
    print('Champion ist bereit fuer Zelle 19 (Risk Engine).')
    # Metadaten anzeigen
    meta_path = os.path.join(SAVE_DIR, 'multiverse_champion_meta.json')
    ChampionPersistence.print_meta(meta_path)
else:
    print('Kein Champion hat alle Szenarien ueberlebt.')
    print('Tipp: MV_MAX_DD erhoehen oder MV_GENERATIONS erhoehen.')

## Zelle 19: Risk Engine — Multiverse Champion zu Live-Trading

Verbindet den Multiverse-Champion (aus Zelle 18) mit dem vollstaendigen
Risk Management System: 1%-Regel, Kelly-Ceiling, ATR-Stop-Loss,
Circuit-Breaker, Drawdown-Limit.

Der Champion kann auch direkt von Drive geladen werden (kein Neustart noetig).
Fuehrt eine simulierte TradingSession durch und zeigt den vollstaendigen Report.

In [ ]:
# ===== RISK ENGINE - Multiverse Champion zu Live-Trading-Session =====
import importlib.util, sys, os

PROJECT_DIR = '/content/BITCOIN4Traders'
SAVE_DIR = '/content/drive/MyDrive/BITCOIN4Traders/data'

# Risk Engine laden
_risk_path = os.path.join(PROJECT_DIR, 'risk_engine.py')
_spec = importlib.util.spec_from_file_location('risk_engine', _risk_path)
_risk = importlib.util.module_from_spec(_spec)
_spec.loader.exec_module(_risk)
sys.modules['risk_engine'] = _risk

# Darwin Engine laden (fuer ChampionPersistence)
if 'darwin_engine' not in sys.modules:
    _darwin_path = os.path.join(PROJECT_DIR, 'darwin_engine.py')
    _spec2 = importlib.util.spec_from_file_location('darwin_engine', _darwin_path)
    _darwin = importlib.util.module_from_spec(_spec2)
    _spec2.loader.exec_module(_darwin)
    sys.modules['darwin_engine'] = _darwin

from risk_engine import run_full_pipeline, RiskConfig
from darwin_engine import ChampionPersistence

# ===== OPTION A: Champion aus Zelle 18 nutzen =====
# (funktioniert wenn Zelle 18 in dieser Session ausgefuehrt wurde)
champion_to_use = globals().get('multiverse_champion', None)

# ===== OPTION B: Champion von Drive laden (nach Neustart) =====
if champion_to_use is None:
    print('Kein Champion in Session - lade von Drive...')
    champion_to_use = ChampionPersistence.load(
        os.path.join(SAVE_DIR, 'multiverse_champion.pkl'),
        os.path.join(SAVE_DIR, 'multiverse_champion_meta.json'),
    )

if champion_to_use is None:
    print('Kein gespeicherter Champion gefunden. Bitte zuerst Zelle 18 ausfuehren.')
else:
    print(f'Champion geladen: {champion_to_use.name} ({type(champion_to_use).__name__})')

    # ===== RISK-KONFIGURATION =====
    risk_cfg = RiskConfig(
        initial_capital=10_000.0,   # Startkapital in USD
        risk_per_trade=0.01,        # 1%-Regel: max 1% pro Trade riskieren
        kelly_fraction=0.5,         # Half-Kelly (konservativ)
        rr_ratio=2.0,               # Risk-Reward Ratio (1:2)
        atr_sl_multiplier=2.0,      # Stop-Loss = Entry +/- 2x ATR
        fee_rate=0.001,             # 0.1% Handelsgebuehr (Binance Taker)
        slippage_rate=0.0005,       # 0.05% Slippage
        max_daily_loss_pct=0.05,    # Circuit-Breaker: >5% Tagesverlust -> Stopp
        max_consecutive_losses=3,   # Circuit-Breaker: 3 Verluste hintereinander
        max_drawdown_pct=0.15,      # Drawdown-Limit: >15% -> Stopp
        min_capital_pct=0.30,       # Min-Kapital-Gate: <30% Startkapital
    )

    # ===== PIPELINE STARTEN =====
    # Tournament -> LiveTradingGuard (6 Gates) -> RiskEngine -> TradingSession
    session_report = run_full_pipeline(
        symbol='BTC/USDT',
        timeframe='1h',
        n_bars=1500,
        risk_config=risk_cfg,
        n_generations=5,
        population_size=16,
        n_wfv_splits=3,
    )

    if session_report is not None:
        session_report.print_summary()
    else:
        print('Pipeline gestoppt - kein Champion hat alle 6 Sicherheits-Gates bestanden.')
        print('Kein Trade ist besser als ein schlechter Trade.')